In [ ]:
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm
import accelerate
from IPython.display import clear_output

import os
os.environ["HUGGINGFACE_TOKEN"] = "your token"
from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"])

In [ ]:
# posts_llama_2000 = pd.read_csv("/kaggle/input/data-for-llama-labeling/data_llama_raw.csv")
posts_llama_2000

In [ ]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"

max_memory_map = {
    0: "14GiB", 
    1: "14GiB"
}

print("Загрузка токенизатора...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)

print("Загрузка модели с device_map='auto' и max_memory...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    max_memory=max_memory_map,
)
clear_output()
print("Модель загружена.")

In [ ]:
system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.
Верни ТОЛЬКО ОДНО ЧИСЛО: 1 (покупка/бычий), 0 (нейтральный), -1 (продажа/медвежий). Без пояснений.

Вот примеры:

Пост: "Отличный отчет! Ракета готова к взлету, закупаюсь на всю котлету #SBER"
Ответ: 1
Пост: "Фиксанул прибыль по $GAZP. Рынок выглядит перегретым, возможна коррекция."
Ответ: -1
Пост: "Сегодня ВТБ опубликует финансовые результаты за квартал. Интересно посмотреть."
Ответ: 0
Пост: "Что думаете про $SBER? Вроде и дивы хорошие, но геополитика давит..."
Ответ: 0
Пост: "Зашортил сбер перед отчетом"
Ответ: -1
Пост: "пора докупать сбербанк пока дешевый"
Ответ: 1
Пост: "какие прогнозы по втб?"
Ответ: 0

Теперь твоя задача: проанализируй следующий пост и верни ТОЛЬКО число.
"""

# query = "пора сливать это дерьмо"

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
]


input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt" # Возвращаем тензоры PyTorch
)

input_ids = input_ids.to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print(f"\nЗапрос к модели:\nSystem: {system_prompt}\nUser: {query}")
print("\nГенерация ответа...")


outputs = model.generate(
    input_ids,
    max_new_tokens=20,
    eos_token_id=terminators,
    do_sample=False, 
    pad_token_id=tokenizer.eos_token_id 
)

response_ids = outputs[0][input_ids.shape[1]:]
response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

print(f"\nСгенерированный ответ модели (сырой):")
print(response_text)

try:
    sentiment_score = int(response_text.strip())
    print(f"\nИзвлеченное значение сентимента: {sentiment_score}")
except ValueError:
    print(f"\nНе удалось извлечь число из ответа: '{response_text.strip()}'")
    print("Возможно, модель сгенерировала дополнительный текст. Попробуйте настроить промпт или параметры генерации.")

In [ ]:
response_text

In [ ]:
def generate_response(system_prompt, query):
    # Format input with system prompt and user query
    prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{query}\n<|assistant|>"
    
    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        inputs.input_ids,
        max_new_tokens=10,  # Small value since we only need a single number
        # temperature=0.1,    # Low temperature for consistent results
        do_sample=False     # Deterministic generation
    )
    
    # Decode the response, removing the input prompt
    response = tokenizer.decode(output[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    return response

# Generate sentiment analysis result
sentiment_result = generate_response(system_prompt, query)
print(f"Сентимент публикации: {sentiment_result}")

In [ ]:
sentiment_result

In [ ]:
messages = [
    {"role": "system", "content": f"{system_prompt}"},
    {"role": "user", "content": f"{query}"},
]

outputs = pipeline(
    messages,
    max_new_tokens=10,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
print("Генерация ответа...")
outputs = model.generate(
    input_ids,
    max_new_tokens=10,  # Оставляем мало токенов для короткого ответа
    # Опционально: параметры для улучшения качества генерации (можно раскомментировать и настроить)
    # temperature=0.1,
    # top_k=50,
    # top_p=0.95,
    # do_sample=True,
    pad_token_id=tokenizer.eos_token_id # Явно указываем pad_token_id, если он есть
)

# 5. Декодируйте только сгенерированную часть ответа
generated_token_ids = outputs[0][input_ids.shape[-1]:]

In [ ]:

from tqdm.auto import tqdm 
import time # Для небольшой паузы и избежания перегрева/ошибок

In [ ]:
def classify_sentiment_llama(
    df: pd.DataFrame,
    model, # Тип: AutoModelForCausalLM
    tokenizer, # Тип: AutoTokenizer
    system_prompt: str,
    text_column: str = "processed_posts",
    output_column: str = "llama_sentiment",
    max_new_tokens: int = 10,
    default_sentiment: int = 0, # Значение по умолчанию при ошибке
    batch_size: int = 1 # Обработка по одному посту (проще для начала)
                       # Батчинг > 1 требует более сложной логики паддинга
) -> pd.DataFrame:
    """
    Классифицирует сентимент постов в DataFrame с помощью модели Llama.

    Args:
        df: Входной DataFrame.
        model: Загруженная модель Llama (AutoModelForCausalLM).
        tokenizer: Загруженный токенизатор Llama (AutoTokenizer).
        system_prompt: Системный промпт для модели.
        text_column: Имя колонки с текстом постов.
        output_column: Имя новой колонки для сохранения результата.
        max_new_tokens: Максимальное количество генерируемых токенов.
        default_sentiment: Значение, присваиваемое в случае ошибки парсинга ответа.
        batch_size: Размер батча (пока поддерживается только 1).

    Returns:
        DataFrame с добавленной колонкой output_column.
    """
    if batch_size != 1:
        raise NotImplementedError("Batch size > 1 is not yet implemented in this function.")

    results = []
    
    # Определяем токены остановки один раз
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # Итерация по постам с использованием tqdm для прогресс-бара
    print(f"Начинаем обработку {len(df)} постов...")
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Классификация постов"):
        post_text = row[text_column]
        
        # Проверка на пустой или некорректный текст
        if not isinstance(post_text, str) or not post_text.strip():
            print(f"Предупреждение: Пропущен пустой или некорректный пост с индексом {index}")
            results.append(default_sentiment)
            continue

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": post_text}
        ]

        try:
            # 1. Токенизация
            input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device) # Сразу на устройство модели

            # 2. Генерация
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                eos_token_id=terminators,
                do_sample=False, # Важно для детерминированного вывода
                pad_token_id=tokenizer.eos_token_id 
            )
            
            # 3. Декодирование только сгенерированной части
            response_ids = outputs[0][input_ids.shape[1]:]
            response_text = tokenizer.decode(response_ids, skip_special_tokens=True).strip()

            # 4. Попытка извлечь число
            try:
                sentiment_score = int(response_text)
                # Опционально: проверка, что число в допустимом диапазоне
                if sentiment_score not in [-1, 0, 1]:
                     print(f"Предупреждение: Модель вернула некорректное число {sentiment_score} для поста {index}. Ответ: '{response_text}'. Используется значение по умолчанию {default_sentiment}.")
                     results.append(default_sentiment)
                else:
                    results.append(sentiment_score)
            except ValueError:
                # Если модель вернула не число
                print(f"Предупреждение: Не удалось извлечь число из ответа для поста {index}. Ответ: '{response_text}'. Используется значение по умолчанию {default_sentiment}.")
                results.append(default_sentiment)

        except Exception as e:
            # Обработка других ошибок (например, OOM при генерации)
            print(f"Ошибка при обработке поста {index}: {e}")
            print(f"Текст поста: {post_text[:100]}...") # Показать начало поста
            results.append(default_sentiment) # Добавляем значение по умолчанию
            # Можно добавить небольшую паузу, если ошибки связаны с перегрузкой
            # time.sleep(1) 

    # Добавляем результаты как новую колонку
    df[output_column] = results
    print("Обработка завершена.")
    return df

# --- Пример вызова функции ---
# Убедитесь, что model и tokenizer загружены и posts_llama_2000 существует

# Копируем DataFrame, чтобы не изменять оригинал (хорошая практика)
df_processed = posts_llama_2000.copy() 

# Запускаем классификацию
df_processed = classify_sentiment_llama(
    df=df_processed,
    model=model,
    tokenizer=tokenizer,
    system_prompt=system_prompt,
    text_column="processed_posts", # Убедитесь, что имя колонки правильное
    output_column="llama_3_1_sentiment" # Новое имя колонки
)

# --- Проверка результатов ---
print("\nПример результатов:")
print(df_processed.head())

print("\nРаспределение полученных классов:")
print(df_processed['llama_3_1_sentiment'].value_counts()) 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import clear_output
import time # Опционально, для проверки времени

In [ ]:
MODEL_NAME = "Qwen/Qwen3-14B" 
# Указываем память, хотя модель должна поместиться и без offload
max_memory_map = {
    0: "14GiB", 
    1: "14GiB"
    # "cpu": "..." # Можно добавить, но скорее всего не понадобится
}

# --- Загрузка ---
print(f"Загрузка токенизатора: {MODEL_NAME}")
# Добавляем trust_remote_code=True, т.к. модели Qwen могут его требовать
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) 

print("Загрузка модели...")
# Используем torch_dtype="auto" для FP8
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto", # Рекомендовано для FP8
    max_memory=max_memory_map,
    trust_remote_code=True # Важно для Qwen
)
clear_output()
print("Модель Qwen3-14B загружена.")

In [14]:
system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.
Верни ТОЛЬКО ОДНО ЧИСЛО: 1 (покупка/бычий), 0 (нейтральный), -1 (продажа/медвежий)

Вот примеры:

Пост: "Отличный отчет! Ракета готова к взлету, закупаюсь на всю котлету #AAPL"
Ответ: 1

Пост: "Фиксанул прибыль по $GAZP. Рынок выглядит перегретым, возможна коррекция."
Ответ: -1

Пост: "Сегодня ВТБ опубликует финансовые результаты за квартал. Интересно посмотреть."
Ответ: 0

Пост: "Что думаете про $SBER? Вроде и дивы хорошие, но геополитика давит..."
Ответ: 0

Пост: "Зашортил сбер перед отчетом"
Ответ: -1

Пост: "пора докупать сбербанк пока дешевый"
Ответ: 1

Пост: "какие прогнозы по втб?"
Ответ: 0

Теперь твоя задача: проанализируй следующий пост и верни ТОЛЬКО число.
"""

# query = "беру сбер в шорт"

# # --- Подготовка ввода для модели Qwen ---
# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": query}
# ]

In [21]:
system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.

Категории и правила:
*   **1 (Покупка/Бычий):** Пост явно выражает намерение купить, решение о покупке, удержание лонга с ожиданием роста, или описывает сильные позитивные факторы, прямо указывающие на вероятный рост цены акции. Ключевые слова/идеи: "купил", "докупаю", "лонг", "ракета", "рост", "пробой вверх", "отчет супер", "пора брать", "держать дальше", "потенциал есть".
*   **-1 (Продажа/Медвежий):** Пост явно выражает намерение продать, решение о продаже, открытие/удержание шорта с ожиданием падения, или описывает сильные негативные факторы, прямо указывающие на вероятное падение цены акции. Ключевые слова/идеи: "продал", "шорт", "сливаю", "падение", "дно не найдено", "отчет плохой", "пора выходить", "фиксация убытка", "коррекция".
*   **0 (Нейтральный):** Пост НЕ содержит явного торгового сигнала на покупку или продажу. Сюда относятся:
    *   Вопросы о цене или прогнозах ("что думаете?", "куда пойдет?").
    *   Констатация фактов или новостей без явной оценки влияния на цену ("вышел отчет", "сегодня дивгэп").
    *   Смешанные сигналы или сомнения ("вроде растет, но страшно", "с одной стороны..., с другой...").
    *   Общие рыночные рассуждения без привязки к конкретному действию.
    *   Фиксация прибыли или убытка *без явного прогноза* дальнейшего движения ("закрыл позицию", "вышел в ноль").
    *   Неясные или неинформативные сообщения.

Примеры:
Пост: "Отличный отчет! Ракета готова к взлету, закупаюсь на всю котлету #AAPL"
Ответ: 1
Пост: "Фиксанул прибыль по $GAZP. Рынок выглядит перегретым, возможна коррекция."
Ответ: -1
Пост: "Сегодня ВТБ опубликует финансовые результаты за квартал. Интересно посмотреть."
Ответ: 0
Пост: "Что думаете про $SBER? Вроде и дивы хорошие, но геополитика давит..."
Ответ: 0
Пост: "Зашортил сбер перед отчетом"
Ответ: -1
Пост: "пора докупать сбербанк пока дешевый"
Ответ: 1
Пост: "какие прогнозы по втб?"
Ответ: 0
Пост: "Закрыл лонг по Лукойлу в плюс, пока понаблюдаю со стороны"
Ответ: 0

Верни ТОЛЬКО ОДНО число: 1, 0 или -1.

Проанализируй следующий пост пользователя и предоставь числовой ответ:
"""

In [25]:
system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.
Верни ТОЛЬКО ОДНО ЧИСЛО: 1 (покупаем актив/бычий), 0 (нейтральный), -1 (продаем актив /медвежий)

Проанализируй следующий пост и верни ТОЛЬКО число."""

In [ ]:


# Применяем шаблон чата Qwen, отключаем "thinking"
# tokenize=True вернет input_ids и attention_mask сразу
model_inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    enable_thinking=False, # Отключаем режим размышления для простой задачи
    return_tensors="pt"
).to(model.device)

print(f"\nЗапрос к модели:\nSystem: {system_prompt}\nUser: {query}")
print("\nГенерация ответа...")
start_time = time.time()

# --- Генерация ответа ---
# Нет необходимости явно указывать eos_token_id при enable_thinking=False и do_sample=False, 
# модель должна остановиться сама или на max_new_tokens
outputs = model.generate(
    model_inputs, # Передаем input_ids и attention_mask (если есть)
    max_new_tokens=10, # Мало токенов для простого ответа
    do_sample=False,   # Детерминированный вывод
    pad_token_id=tokenizer.eos_token_id 
)

end_time = time.time()
print(f"Время генерации: {end_time - start_time:.2f} сек")

# --- Декодирование и вывод ---
# Декодируем только сгенерированную часть
# model_inputs может быть словарем, используем input_ids
input_ids_len = model_inputs['input_ids'].shape[1] if isinstance(model_inputs, dict) else model_inputs.shape[1]
response_ids = outputs[0][input_ids_len:] 
response_text = tokenizer.decode(response_ids, skip_special_tokens=True).strip()

print(f"\nСгенерированный ответ модели (сырой):")
print(response_text)

# Попытка извлечь число
try:
    sentiment_score = int(response_text)
    if sentiment_score not in [-1, 0, 1]:
        print(f"\nПредупреждение: Модель вернула некорректное число {sentiment_score}. Используется 0.")
        sentiment_score = 0
    print(f"\nИзвлеченное значение сентимента: {sentiment_score}")
except ValueError:
    print(f"\nНе удалось извлечь число из ответа: '{response_text}'. Используется 0.")
    sentiment_score = 0 # Значение по умолчанию

In [6]:
posts_qwen_2000 = pd.read_csv("/kaggle/input/qwen-dataset/data_qwen_raw.csv")
posts_qwen_2000

,post_id,processed_posts
0,78592a62-93f9-4940-b914-a6cb8baafaff,🪓 {$SGZH} — полностью исключают из основного и...
1,47ed3f81-fead-4d6e-9186-d2b21de56e4c,На этот раз таких активов оказалось неожиданно...
2,01d56071-766d-40a1-801c-4572e703fdc6,"Market Power 🗣 про {$SBER}:\n\n""Сбер превосход..."
3,761e88a9-0b4e-4a46-b569-ca92762b78b4,•29.08.2023 {$SGZH} \nСегежа Групп опубликует ...
4,66369560-83a0-4456-80de-2ba0239b4e1f,"{$SBER}\nЦБ готов идти на более жесткие меры, ..."
...,...,...
2995,c91e9965-9574-463d-86f2-70d472d4f3a9,{$LKOH} \n‼️Давайте сравним акции номер один 1...
2996,e88390b2-a5bb-4403-b6bc-4ac5a8dc2eb5,{$SBER} )))
2997,faf7264b-67fe-4d5d-998c-9e2c026f3b76,{$SBER} {$LKOH} {$T} {$MGNT} {$VTBR} {$GMKN} \...
2998,16c1315f-d62e-4412-8faa-48542fe4b437,"{$SGZH} \n﻿Сдувается мыльный пузырь, что и тре..."


In [ ]:
torch.cuda.empty_cache() 
import gc
gc.collect()

In [28]:
def classify_sentiment_qwen_batch( # Немного изменил имя для ясности
    df: pd.DataFrame,
    model, 
    tokenizer, 
    system_prompt: str,
    text_column: str = "processed_posts",
    output_column: str = "qwen_sentiment", # Имя колонки по умолчанию
    max_new_tokens: int = 10,
    default_sentiment: int = 0, 
    batch_size: int = 1 # Оставляем 1 для простоты, т.к. батчинг требует паддинга
) -> pd.DataFrame:
    """
    Классифицирует сентимент постов в DataFrame с помощью модели Qwen.
    Использует метод передачи результата apply_chat_template напрямую в generate.

    Args:
        df: Входной DataFrame.
        model: Загруженная модель Qwen.
        tokenizer: Загруженный токенизатор Qwen.
        system_prompt: Системный промпт для модели.
        text_column: Имя колонки с текстом постов.
        output_column: Имя новой колонки для сохранения результата.
        max_new_tokens: Максимальное количество генерируемых токенов.
        default_sentiment: Значение, присваиваемое в случае ошибки парсинга ответа.
        batch_size: Размер батча (пока поддерживается только 1).

    Returns:
        DataFrame с добавленной колонкой output_column.
    """
    if batch_size != 1:
        raise NotImplementedError("Batch size > 1 requires padding and attention mask handling, currently not implemented here.")

    results = []
    
    print(f"Начинаем обработку {len(df)} постов с моделью Qwen...")
    start_total_time = time.time()
    
    # Итерация по постам с использованием tqdm для прогресс-бара
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Классификация Qwen"):
        post_text = row[text_column]
        
        # Проверка на пустой или некорректный текст
        if not isinstance(post_text, str) or not post_text.strip():
            results.append(default_sentiment)
            continue

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": post_text}
        ]

        try:
            # 1. Токенизация (сразу на GPU)
            #    Эта часть теперь основана на вашем рабочем варианте
            torch.cuda.empty_cache() 
            model_inputs = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                enable_thinking=False, 
                return_tensors="pt"
            ).to(model.device)
            
            # Определяем длину входных данных ДО генерации
            # Эта логика для определения input_ids_len теперь здесь
            if isinstance(model_inputs, dict):
                input_ids_len = model_inputs['input_ids'].shape[1]
            elif hasattr(model_inputs, 'shape'): # Проверка на тензор или подобный объект
                 input_ids_len = model_inputs.shape[1]
            else:
                # Неожиданный тип, пытаемся угадать или пропускаем
                print(f"Предупреждение: Не удалось определить input_ids_len для поста {index}. Тип: {type(model_inputs)}. Пропуск.")
                results.append(default_sentiment)
                continue

            # 2. Генерация (передаем model_inputs напрямую)
            outputs = model.generate(
                model_inputs, # Передаем результат токенизации напрямую
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id 
            )
            # 3. Декодирование
            response_ids = outputs[0][input_ids_len:] # Используем ранее вычисленную длину
            response_text = tokenizer.decode(response_ids, skip_special_tokens=True).strip()

            # 4. Парсинг ответа
            try:
                sentiment_score = int(response_text)
                if sentiment_score not in [-1, 0, 1]:
                     # print(f"Предупр: Некорр. число {sentiment_score} для поста {index}. Ответ: '{response_text}'.") # Отладка
                     results.append(default_sentiment)
                else:
                    results.append(sentiment_score)
            except ValueError:
                # print(f"Предупр: Не число для поста {index}. Ответ: '{response_text}'.") # Отладка
                results.append(default_sentiment)

        except Exception as e:
            # Обработка других ошибок (например, OOM при генерации)
            print(f"Критическая ошибка при обработке поста {index}: {e}")
            print(f"Текст поста: {post_text[:100]}...") 
            results.append(default_sentiment) # Добавляем значение по умолчанию
            # time.sleep(1) # Можно добавить паузу при частых ошибках

    # Добавляем результаты как новую колонку
    df[output_column] = results
    
    end_total_time = time.time()
    total_time = end_total_time - start_total_time
    avg_time = total_time / len(df) if len(df) > 0 else 0
    print(f"Обработка завершена за {total_time:.2f} сек ({avg_time:.3f} сек/пост).")
    return df

# --- Пример вызова функции ---
# Убедитесь, что model (например, Qwen/Qwen3-14B-Instruct) и tokenizer загружены
# и DataFrame posts_llama_2000 существует

df_for_qwen_processing = posts_qwen_2000.copy() 

df_classified_qwen = classify_sentiment_qwen_batch(
    df=df_for_qwen_processing,
    model=model, 
    tokenizer=tokenizer, 
    system_prompt=system_prompt,
    text_column="processed_posts", 
    output_column="qwen_14b_instruct_sentiment" # Укажите желаемое имя колонки
)

# --- Проверка результатов ---
print("\nПример результатов Qwen:")
print(df_classified_qwen.head())

print("\nРаспределение полученных классов Qwen:")
print(df_classified_qwen['qwen_14b_instruct_sentiment'].value_counts()) 

Начинаем обработку 3000 постов с моделью Qwen...


Классификация Qwen:   0%|          | 0/3000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Классификация Qwen: 100%|██████████| 300

Обработка завершена за 2333.41 сек (0.778 сек/пост).

Пример результатов Qwen:
                                post_id  \
0  78592a62-93f9-4940-b914-a6cb8baafaff   
1  47ed3f81-fead-4d6e-9186-d2b21de56e4c   
2  01d56071-766d-40a1-801c-4572e703fdc6   
3  761e88a9-0b4e-4a46-b569-ca92762b78b4   
4  66369560-83a0-4456-80de-2ba0239b4e1f   

                                     processed_posts  \
0  🪓 {$SGZH} — полностью исключают из основного и...   
1  На этот раз таких активов оказалось неожиданно...   
2  Market Power 🗣 про {$SBER}:\n\n"Сбер превосход...   
3  •29.08.2023 {$SGZH} \nСегежа Групп опубликует ...   
4  {$SBER}\nЦБ готов идти на более жесткие меры, ...   

   qwen_14b_instruct_sentiment  
0                           -1  
1                            0  
2                            1  
3                            0  
4                           -1  

Распределение полученных классов Qwen:
qwen_14b_instruct_sentiment
 0    1050
-1    1024
 1     926
Name: count, dtype: int64


In [29]:
df_classified_qwen

,post_id,processed_posts,qwen_14b_instruct_sentiment
0,78592a62-93f9-4940-b914-a6cb8baafaff,🪓 {$SGZH} — полностью исключают из основного и...,-1
1,47ed3f81-fead-4d6e-9186-d2b21de56e4c,На этот раз таких активов оказалось неожиданно...,0
2,01d56071-766d-40a1-801c-4572e703fdc6,"Market Power 🗣 про {$SBER}:\n\n""Сбер превосход...",1
3,761e88a9-0b4e-4a46-b569-ca92762b78b4,•29.08.2023 {$SGZH} \nСегежа Групп опубликует ...,0
4,66369560-83a0-4456-80de-2ba0239b4e1f,"{$SBER}\nЦБ готов идти на более жесткие меры, ...",-1
...,...,...,...
2995,c91e9965-9574-463d-86f2-70d472d4f3a9,{$LKOH} \n‼️Давайте сравним акции номер один 1...,1
2996,e88390b2-a5bb-4403-b6bc-4ac5a8dc2eb5,{$SBER} ))),0
2997,faf7264b-67fe-4d5d-998c-9e2c026f3b76,{$SBER} {$LKOH} {$T} {$MGNT} {$VTBR} {$GMKN} \...,0
2998,16c1315f-d62e-4412-8faa-48542fe4b437,"{$SGZH} \n﻿Сдувается мыльный пузырь, что и тре...",-1


In [17]:
df_classified_qwen.processed_posts.values

array(['🪓 {$SGZH} — полностью исключают из основного индекса Мосбиржи. Ее место займет 📱 $YDEX.',
       'На этот раз таких активов оказалось неожиданно много, ведь осень обычно не является заметным дивидендным сезоном. До конца года около 20 эмитентов, в том числе {$LKOH} ЛУКОЙЛ, {$RTKM} Ростелеком, {$GMKN} Норникель и {$PHOR} Фосагро, планируют выплатить дивиденд.',
       'Market Power 🗣 про {$SBER}:\n\n"Сбер превосходит себя\nБольшой зеленый банк опубликовал отчет за 2 квартал и 1 полугодие\n\nСбер\nМСар = ₽6 трлн\n\n📊Итоги 2 квартала\n- процентные доходы: ₽598 млрд;\n- комиссионные доходы: ₽187 млрд;\n- чистая прибыль: ₽380 млрд;\n- рентабельность капитала: 26%;\n- активные физлица: 107 млн (+1% с начала года);\n- ежемесячные пользователи СберОнлайн: 80 млн (+1,5% с начала года);\n- розничный кредитный портфель: ₽14 трлн (+12% с начала года);\n- ипотечный портфель: ₽8,5 трлн (+12% с начала года).\n\n ❗️Стоит отметить, что прибыль за 2 квартал оказалась даже выше консенсус-прогнозо

In [ ]:
# !pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig # Импортируем конфиг

MODEL_NAME = "Qwen/Qwen3-14B" 

# Конфигурация для 4-битной квантизации (NF4)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.bfloat16, # Тип для вычислений (рекомендуется bfloat16)
#     bnb_4bit_quant_type="nf4",           # Тип квантизации
#     bnb_4bit_use_double_quant=True,     # Использовать двойную квантизацию для экономии
# )

# Или конфигурация для 8-битной квантизации (если 4-битная дает плохое качество)
quantization_config = BitsAndBytesConfig(
   load_in_8bit=True,
)

print(f"Загрузка токенизатора: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

print("Загрузка квантизированной модели...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto", # device_map="auto" хорошо работает с bitsandbytes
    quantization_config=quantization_config, # Применяем конфиг квантизации
    trust_remote_code=True 
    # max_memory можно убрать или оставить, но теперь модель должна легко помещаться
)
print("Квантизированная модель загружена.")